https://github.com/pushshift/api

In [155]:
import pandas as pd

import praw
import urllib.request
import json

from datetime import datetime
import time


reddit = praw.Reddit(
    client_id="kRAJROlrIHaWMw",
    client_secret="f1EXDzFpTw1AryR_3urQ9O6I3AQ",
    user_agent="AITA_scrape",
)

In [4]:
def return_pushshift_data(subreddit, early_before, late_after, min_comments, min_score):
    link = (
        "https://api.pushshift.io/reddit/search/submission/?subreddit=%s&sort_type=created_utc&sort=desc&size=1000&before=%ss&after=%ss&num_comments=>%s&score=>%s"
        % (subreddit, early_before, late_after, min_comments, min_score)
    )
    print(link)
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read())
    data = data["data"]
    return data


def unix_to_date(unix):
    str_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(unix))
    date_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M:%S")
    return date_time


def pushshift_reddit_search(subreddit, min_comments, min_score, years_ago):
    early_before = 0
    late_after = years_ago * 365 * 60 * 60 * 24
    matches = 1000
    posts = []
    while matches >= 1000:
        data = return_pushshift_data(
            subreddit, early_before, late_after, min_comments, min_score
        )
        matches = len(data)

        for match in range(0, matches):
            posts.append(
                [
                    data[match]["title"],
                    data[match]["id"],
                    data[match]["score"],
                    data[match]["num_comments"],
                    data[match]["selftext"],
                    unix_to_date(data[match]["created_utc"]),
                ]
            )

        if matches == 1000:
            unix_time = int(data[matches - 1]["created_utc"])
            last_time = unix_to_date(unix_time)
            now = unix_to_date(time.time())
            early_before = int((now - last_time).total_seconds() + 60)

    post_df = pd.DataFrame(
        posts, columns=["title", "id", "score", "num_comments", "body", "date"],
    )

    return post_df


post_df = pushshift_reddit_search(
    subreddit="AmItheAsshole", min_comments=20, min_score=50, years_ago=2
)

len(post_df)

https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=0s&after=63072000s&num_comments=>20&score=>50
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=14092987s&after=63072000s&num_comments=>20&score=>50
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=15121701s&after=63072000s&num_comments=>20&score=>50
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=16132479s&after=63072000s&num_comments=>20&score=>50
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&before=17184485s&after=63072000s&num_comments=>20&score=>50
https://api.pushshift.io/reddit/search/submission/?subreddit=AmItheAsshole&sort_type=created_utc&sort=desc&size=1000&befor

21014

In [85]:
import collections

print(
    [
        item
        for item, count in collections.Counter(post_df["id"].tolist()).items()
        if count > 1
    ]
)

[]


In [20]:
def grab_flairs_from_subid(post_df):
    flairs = []
    count = 0
    start = datetime.now()
    id_list = list(post_df["id"])

    for n in range(0, len(id_list)):
        post_id = id_list[n]
        flair = reddit.submission(id=post_id).link_flair_text
        flairs.append(flair)
        count += 1
        if count % 2000 == 0:
            end = datetime.now()
            time = end - start
            print("Parsed: %s (%s)" % (count, time))
            start = datetime.now()

    return flairs


flair_list = grab_flairs_from_subid(post_df)

Parsed: 2000 (0:56:52.039695)
Parsed: 4000 (0:56:01.669518)
Parsed: 6000 (0:55:54.824723)
Parsed: 8000 (0:59:37.410303)
Parsed: 10000 (0:56:49.584845)
Parsed: 12000 (0:53:02.799520)
Parsed: 14000 (0:51:45.980991)
Parsed: 16000 (0:52:14.788159)
Parsed: 18000 (0:48:33.263865)
Parsed: 20000 (0:46:06.515758)


In [22]:
print(len(flair_list))
post_df["verdict"] = flair_list
post_df.to_csv("data_raw/full_post_list.csv")
post_df.head()

21014


,title,id,score,num_comments,body,date,verdict
0,AITA for picking up homeless girls?,fxhdfh,52,163,I'm a second-year radiology resident in a city...,2020-04-08 23:30:21,None
1,UPDATE: AITA for ignoring my brother and not c...,fvj4jw,78,310,\[Original post\] ([https://www.reddit.com/r/...,2020-04-05 18:58:42,UPDATE
2,Aita for throwing away my 33 year old boyfrien...,fshmat,145,1056,Yes I'm aware of how stupid and ridiculous thi...,2020-03-31 17:53:13,None
3,AITA for letting my 12 year old daughter sleep...,fsghbo,84,102,Ok so since my daughter has been old enough to...,2020-03-31 16:53:09,None
4,AITA for leaving a girl's house after she fail...,fsflpf,64,78,[removed],2020-03-31 16:06:11,None
